# Mobile Game Analysis

### An analysis of user progress data

Data can be found on Kaggle at [this link](https://www.kaggle.com/datasets/manchvictor/prediction-of-user-loss-in-mobile-games)

level_seq.csv

Each record is an attempt to play a level. The meaning of each column is as follows:
- 'user_id' : user id, which can be matched with those in training, verification, and test sets;
- 'level_id' : level id;
- f_success ': indicates whether to complete the clearance (1: completes the clearance, 0: fails).
- f_duration ': the duration of the attempt (unit: s);
- f_reststep ': the ratio of the remaining steps to the limited steps (failure is 0);
- f_help ': Whether extra help, such as props and hints, was used (1: used, 0: not used);
'time' : indicates the timestamp.

level_meta.csv

Some statistical characteristics of each level can be used to represent the level. The meaning of each column is as follows:

- f_avg_duration ': Average time spent on each attempt (unit s, including successful and failed attempts);
- 'f_avg_passrate' : average clearance rate;
- f_avg_win_duration ': Average time spent on each clearance (in s, including only the attempts to clear the clearance);
- f_avg_retrytimes' : Average number of retries (the second time to play the same level counts as the first retry);
- 'level_id' : indicates the id of the level, which can be matched with the level in level_seq.csv.

## Step 1: Import and Cleaning

In [2]:
#Import libraries
import pandas as pd
import glob
import os
import datetime as dt
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import matplotlib.dates
import matplotlib.dates as mdates
import seaborn as sns

In [3]:
#Import csv files
path = "/Users/raws/Downloads/mobile_game_data"
csv_files = glob.glob(path + "/*.csv")

csv_files

['/Users/raws/Downloads/mobile_game_data/test.csv',
 '/Users/raws/Downloads/mobile_game_data/level_seq.csv',
 '/Users/raws/Downloads/mobile_game_data/dev.csv',
 '/Users/raws/Downloads/mobile_game_data/train.csv',
 '/Users/raws/Downloads/mobile_game_data/level_meta.csv']

In [4]:
#Append csvs to list using list comprehension
df_list = [pd.read_csv(filename, delimiter='\t', index_col=None, header=0) for filename in csv_files]
df_list

[      user_id
 0           1
 1           2
 2           3
 3           4
 4           5
 ...       ...
 2768     2769
 2769     2770
 2770     2771
 2771     2772
 2772     2773
 
 [2773 rows x 1 columns],
          user_id  level_id  f_success  f_duration  f_reststep  f_help  \
 0          10932         1          1       127.0    0.500000       0   
 1          10932         2          1        69.0    0.703704       0   
 2          10932         3          1        67.0    0.560000       0   
 3          10932         4          1        58.0    0.700000       0   
 4          10932         5          1        83.0    0.666667       0   
 ...          ...       ...        ...         ...         ...     ...   
 2194346    10931        40          1       111.0    0.250000       1   
 2194347    10931        41          1        76.0    0.277778       0   
 2194348    10931        42          0       121.0    0.000000       1   
 2194349    10931        42          0       115.0  

In [29]:
users = df_list[1]
levels = df_list[4]

In [25]:
users.head()

,user_id,level_id,f_success,f_duration,f_reststep,f_help,time
0,10932,1,1,127.00,0.50,0,2020-02-01 00:05:51
1,10932,2,1,69.00,0.70,0,2020-02-01 00:08:01
2,10932,3,1,67.00,0.56,0,2020-02-01 00:09:50
3,10932,4,1,58.00,0.70,0,2020-02-01 00:11:16
4,10932,5,1,83.00,0.67,0,2020-02-01 00:13:12


In [7]:
#Suppress scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)
users.describe()

,user_id,level_id,f_success,f_duration,f_reststep,f_help
count,2194351.00,2194351.00,2194351.00,2194351.00,2194351.00,2194351.00
mean,6745.03,96.84,0.53,108.12,0.17,0.04
std,3942.09,84.11,0.50,53.61,0.23,0.21
min,1.00,1.00,0.00,1.00,0.00,0.00
25%,3287.00,41.00,0.00,77.00,0.00,0.00
50%,6688.00,80.00,1.00,100.00,0.05,0.00
75%,10163.00,142.00,1.00,127.00,0.29,0.00
max,13589.00,1509.00,1.00,600.00,1.00,1.00


In [10]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2194351 entries, 0 to 2194350
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     int64  
 1   level_id    int64  
 2   f_success   int64  
 3   f_duration  float64
 4   f_reststep  float64
 5   f_help      int64  
 6   time        object 
dtypes: float64(2), int64(4), object(1)
memory usage: 117.2+ MB


In [8]:
users.duplicated().sum()

69322

In [35]:
users.isna().sum()

user_id       0
level_id      0
f_success     0
f_duration    0
f_reststep    0
f_help        0
time          0
dtype: int64

In [43]:
for col in users:
    print(users[col].isna().sum())

0
0
0
0
0
0
0


In [9]:
dupes = users[users.duplicated()]
dupes

,user_id,level_id,f_success,f_duration,f_reststep,f_help,time
64,10932,50,0,153.00,0.00,0,2020-02-01 17:05:43
65,10932,50,0,153.00,0.00,0,2020-02-01 17:05:43
665,2774,42,1,130.00,0.05,0,2020-02-01 10:56:34
667,2774,44,1,116.00,0.21,0,2020-02-01 11:02:32
707,2774,62,1,89.00,0.43,0,2020-02-02 09:10:50
...,...,...,...,...,...,...,...
2192592,10924,36,1,110.00,0.50,0,2020-02-02 00:45:41
2193091,13586,104,0,155.00,0.00,0,2020-02-03 21:13:35
2193569,10927,137,1,73.00,0.29,0,2020-02-02 22:36:23
2193666,10927,207,0,166.00,0.00,0,2020-02-04 20:59:36


<div class="alert alert-warning">
  <strong>Summary of Findings</strong>
    <li>Out of roughly 2.2 million rows there are about 70k duplicates.</li>
    <li>Out of roughly 2.2 million rows there are about 70k duplicates.</li>
    <li>user_id needs to be converted to string.</li>
    <li>time needs to be converted to datetime.</li>
</div>

In [11]:
users.drop_duplicates(inplace=True)

In [14]:
users['user_id'] = users['user_id'].astype('str')

In [16]:
users['time'] = pd.to_datetime(users['time'], format='%Y-%m-%d %H:%M:%S')

In [17]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2125029 entries, 0 to 2194350
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     object        
 1   level_id    int64         
 2   f_success   int64         
 3   f_duration  float64       
 4   f_reststep  float64       
 5   f_help      int64         
 6   time        datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 129.7+ MB


<div class="alert alert-success">
  <strong>Summary of Actions</strong>
    <li>Dropped ~70k duplicates.</li>
    <li>Changed user_id to string.</li>
    <li>Converted time to datetime object.</li>
</div>

In [30]:
levels.head()

,f_avg_duration,f_avg_passrate,f_avg_win_duration,f_avg_retrytimes,level_id
0,39.89,0.94,35.58,0.02,1
1,60.68,0.99,56.72,0.00,2
2,76.95,0.99,71.79,0.00,3
3,58.17,0.99,54.84,0.00,4
4,101.78,0.95,85.65,0.03,5


In [31]:
levels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1509 entries, 0 to 1508
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   f_avg_duration      1509 non-null   float64
 1   f_avg_passrate      1509 non-null   float64
 2   f_avg_win_duration  1509 non-null   float64
 3   f_avg_retrytimes    1509 non-null   float64
 4   level_id            1509 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 59.1 KB


In [32]:
levels.describe()

,f_avg_duration,f_avg_passrate,f_avg_win_duration,f_avg_retrytimes,level_id
count,1509.00,1509.00,1509.00,1509.00,1509.00
mean,514.41,0.45,116.01,3.42,755.00
std,234.73,0.19,27.66,4.07,435.76
min,39.89,0.00,35.58,0.00,1.00
25%,372.25,0.33,101.24,2.00,378.00
50%,497.65,0.40,112.67,3.29,755.00
75%,623.59,0.51,126.44,4.41,1132.00
max,2105.00,1.00,540.00,98.00,1509.00


In [33]:
levels.duplicated().sum()

0

## Step 2: EDA

In [21]:
#Examine number of levels
users['level_id'].max()

1509

In [23]:
#Examine success/failure ratio
users['f_success'].value_counts()

1    1121482
0    1003547
Name: f_success, dtype: int64

In [24]:
#Examine level attempt duration time
users['f_duration'].describe()

count   2125029.00
mean        107.97
std          53.58
min           1.00
25%          77.00
50%          99.00
75%         127.00
max         600.00
Name: f_duration, dtype: float64

In [ ]:
users['f_duration']